In [28]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from urllib.request import urlopen

In [68]:
def retrieve_movies_results(url):
    try:
        # On crée une session de navigation web
        session = requests.Session()
        response = session.get(url)
        
        # On récupère les paramètres de cookies
        cookies_dictionary = session.cookies.get_dict()

        # On construire le cookie
        cookie = '; '.join([f'{key}={value}' for key, value in cookies_dictionary.items()])

        # On récupère le contenu dans l'url avec la configuration indiquée dans le header
        USER_AGENT = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:65.0) Gecko/20100101 Firefox/65.0"
        headers = {"user-agent": USER_AGENT, "cookie": cookie}  # Ajouter le cookie au header
        html_text = requests.get(url, headers=headers, timeout=20).text

        return html_text
    except :
        print('La requète a échouée')
        return None

In [65]:
def process_results(html_text):
    # La libraire BeautifulSoup permet de parser le texte que nous avons extrait de la page web
    soup = BeautifulSoup(html_text)
    # On recherche tous les éléments html <div> qui indiquent des divisions vers la classe que nous cherchons à importer 
    #results = soup.find_all('div', {'class' : 'ipc-html-content-inner-div'})
    data = [synopsis.text for synopsis in soup.find_all('div', {'class' : 'ipc-html-content-inner-div'})]
    #titles  = [title.text for title in soup.find_all('h3', {'class' : 'ipc-title__text'})]  Pour importer les titres mais le soucis est que les titres n'ont pas toujours de descriptions ce qui implique un mismatch de longeur des deux listes.
    results = pd.DataFrame({'Synopsis' : data})
                            #'Titre' : titles})
        
    return results

In [85]:
all_movies = pd.read_csv("/Users/iliasrazig/Documents/ENSAE/S1/Infrastructure/Projet/movies_final.csv", nrows= 100)

In [82]:
len(all_movies["title"].unique())

58529

In [78]:
for val in all_movies["title"].unique() :
    print(val)

Toy Story
Jumanji
Grumpier Old Men
Waiting to Exhale
Father of the Bride Part II
Heat
Sabrina
Tom and Huck
Sudden Death
GoldenEye
American President, The
Dracula: Dead and Loving It
Balto
Nixon
Cutthroat Island
Casino
Sense and Sensibility
Four Rooms
Ace Ventura: When Nature Calls
Money Train


In [92]:
def merging(url, movies) :
    synops = []
    for val in movies.unique() :
        try :
            movie = val.replace(" ", "%20")
            complete_url = url+movie
            html_text = retrieve_movies_results(complete_url)
            results_df = process_results(html_text)
            synops.append(results_df['Synopsis'][0])   
        except : # Parfois un film ne comporte pas de description référencée sur IMDB et parfois soucis de matching entre les noms de la base et ceux de IMDB
            synops.append("Description non available")
    match = pd.DataFrame({'Synopsis' : synops,
                                'title' : movies.unique()})
    return match

In [80]:
match

,Synopsis,title
0,A cowboy doll is profoundly threatened and jea...,Toy Story
1,Four teenagers are sucked into a magical video...,Jumanji
2,John and Max resolve to save their beloved bai...,Grumpier Old Men
3,"Based on Terry McMillan's novel, this film fol...",Waiting to Exhale
4,George Banks must deal not only with his daugh...,Father of the Bride Part II
5,A group of high-end professional thieves start...,Heat
6,"As her 16th birthday nears, Sabrina Spellman m...",Sabrina
7,Two best friends witness a murder and embark o...,Tom and Huck
8,A former fireman takes on a group of terrorist...,Sudden Death
9,Years after a friend and fellow 00 agent is ki...,GoldenEye


In [89]:
synopsis_base = merging(url = 'https://www.imdb.com/search/title/?title=', movies = all_movies["title"])

In [90]:
synopsis_base

,Synopsis,title
0,A cowboy doll is profoundly threatened and jea...,Toy Story
1,Four teenagers are sucked into a magical video...,Jumanji
2,John and Max resolve to save their beloved bai...,Grumpier Old Men
3,"Based on Terry McMillan's novel, this film fol...",Waiting to Exhale
4,George Banks must deal not only with his daugh...,Father of the Bride Part II
...,...,...
95,Description non available,"Hate (Haine, La)"
96,A nerdy florist finds his chance for success a...,Shopping
97,A documentary crew from the BBC arrives in L.A...,Heidi Fleiss: Hollywood Madam
98,The accidental shooting of a boy in New York l...,City Hall


In [91]:
synopsis_base.to_csv('descriptions.csv')